In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from utils import load_data,normalize,doublerelu
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
torch.set_printoptions(sci_mode=False)
import time

In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
cuda = torch.cuda.is_available()
weight_decay = 10e-4
epochs = 10001
seed = 165
hidden = 10
lr = 0.0001

In [4]:
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

In [5]:
class GNN1Layer(Module):

    def __init__(self, batch_size, in_features, out_features):
        super(GNN1Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size

        weight1_eye = torch.FloatTensor(torch.eye(in_features, out_features))
        weight1_eye = weight1_eye.reshape((1, in_features, out_features))
        weight1_eye = weight1_eye.repeat(batch_size, 1, 1)
        self.weight1 = Parameter(weight1_eye)
        self.weight2 = Parameter(torch.zeros(batch_size, in_features, out_features))

    def forward(self, input, adj):
        v1 = torch.bmm(input, self.weight1)
        v2 = torch.bmm(torch.bmm(adj, input), self.weight2)
        output = v1 + v2
        return output

In [6]:
class GNN1(nn.Module):

    def __init__(self, batch_size, nfeat, ndim, hidden):
        super(GNN1, self).__init__()

        self.gc1 = GNN1Layer(batch_size, ndim, ndim)

    def forward(self, x, adj, random_indices):
        f = torch.clone(x)
        x = doublerelu(self.gc1(x, adj))
        x = x/x.sum(axis=2).unsqueeze(2) #normalize st sum = 1

        f[0][random_indices, :] = x[0][random_indices, :]
        
        return f

In [7]:
def train(adj,features,labels,random_indices,test_random_indices, test_features):
    
    adj_norm = normalize(adj)
    
    labels = labels - 1
    
    adj = torch.FloatTensor(adj)
    adj_norm = torch.FloatTensor(adj_norm)
    features = torch.FloatTensor(features)
    labels = torch.FloatTensor(labels)
    test_features = torch.FloatTensor(test_features)
    
    model = GNN1(batch_size=adj.shape[0],
                nfeat=adj.shape[1],
                ndim=nb_label,
                hidden=hidden)
    if cuda:
        model.cuda()
        features = features.cuda()
        adj = adj.cuda()
        adj_norm = adj_norm.cuda()
        labels = labels.cuda()
        test_features = test_features.cuda()
    
    # Train model
    t_total = time.time()

    optimizer = optim.Adam(model.parameters(),
                           lr=lr, weight_decay=weight_decay)
    
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(epochs):

        t = time.time()
        model.train()
        optimizer.zero_grad()

        output = model(features, adj_norm, random_indices)
            
        # Mask accuracy
        accuracy = torch.sum(torch.argmax(output[0][random_indices, :],axis=1)==labels[random_indices, :].reshape(1,-1))/labels[random_indices, :].shape[0]
        
        #Test accuracy
        with torch.no_grad():
            test_output = model(test_features, adj_norm, test_random_indices)
            test_accuracy = torch.sum(torch.argmax(test_output[0][test_random_indices, :],axis=1)==labels[test_random_indices, :].reshape(1,-1))/labels[test_random_indices, :].shape[0]
        
        loss = criterion(output[0],labels.reshape(-1).long())

        loss.backward(retain_graph=True)

        optimizer.step()

        if epoch == 0:
            best_loss = loss
            best_output = output
            best_acc = accuracy
            init_acc = accuracy
            test_init_acc = accuracy
            best_test_acc = test_accuracy
            best_test_op = test_output
        else:
            if loss < best_loss:
                best_loss = loss
                best_output = output
                best_acc = accuracy
                best_test_acc = test_accuracy
                best_test_op = test_output

        if epoch % 1000 == 0:
            print('Epoch: {:04d}'.format(epoch + 1),
                  'Train Accuracy: {:.4f}'.format(best_acc.item()),
                  'Test Accuracy: {:.4f}'.format(best_test_acc.item()),
                  'Loss: {:.8f}'.format(best_loss.item()),
                  'time: {:.4f}s'.format(time.time() - t))
            
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    
    return best_loss,best_output, init_acc, best_acc, best_test_op, best_test_acc, test_init_acc

In [8]:
adj,feature,labels = load_data()

#feature = feature - 1
#nb_label = int(max(feature))+1
#featuress = np.eye(nb_label)[np.array(feature,dtype=int).reshape(1,-1)]

In [9]:
label = np.copy(labels)
label = label - 1
nb_label = int(max(label))+1
featuress = np.eye(nb_label)[np.array(label,dtype=int).reshape(1,-1)]

In [10]:
mask_percentage = [0.3]
init = []
final = []
test_init = []
test_final = []
for m in mask_percentage:
    
    features = np.copy(featuress)
    test_features = np.copy(featuress)
    # Masking
    number_of_rows = features[0].shape[0]
    random_indices = np.random.choice(number_of_rows, size=int(m*number_of_rows), replace=False)
    random_rows = features[0][random_indices, :]
    features[0][random_indices, :] = np.tile(np.array([[0.2]]),random_rows.shape)
    
    test_random_indices = np.random.choice(number_of_rows, size=int(0.1*number_of_rows), replace=False)
    test_random_rows = test_features[0][test_random_indices, :]
    test_features[0][test_random_indices, :] = np.tile(np.array([[0.2]]),test_random_rows.shape)
    
    print("\nMasked {}% of nodes\n".format(int(m*100)))
    prev_loss, op, acc, _, test_op, _, test_acc = train(adj,features,labels, random_indices, test_random_indices, test_features)
    init.append(acc.item())
    test_init.append(test_acc.item())
    #print(op)
    loss, op, _, acc, test_op, test_acc, _ = train(adj,op.cpu().detach().numpy(),labels, random_indices, test_random_indices, test_op.cpu().detach().numpy())
    i = 0
    while loss < prev_loss :
        i += 1
        if i >= 2:
            break
        prev_loss = loss
        loss, op, _, acc, test_op, test_acc, _ = train(adj,op.cpu().detach().numpy(),labels, random_indices, test_random_indices, test_op.cpu().detach().numpy())
    final.append(acc.item())
    test_final.append(test_acc.item())
    


Masked 30% of nodes

Epoch: 0001 Train Accuracy: 0.1314 Test Accuracy: 0.1023 Loss: 1.11618054 time: 0.3650s
Epoch: 1001 Train Accuracy: 0.3694 Test Accuracy: 0.3674 Loss: 1.09252489 time: 0.0030s
Epoch: 2001 Train Accuracy: 0.3694 Test Accuracy: 0.3674 Loss: 1.08271003 time: 0.0030s
Epoch: 3001 Train Accuracy: 0.3648 Test Accuracy: 0.3767 Loss: 1.08164322 time: 0.0040s
Epoch: 4001 Train Accuracy: 0.3740 Test Accuracy: 0.3953 Loss: 1.07989132 time: 0.0030s
Epoch: 5001 Train Accuracy: 0.4281 Test Accuracy: 0.4744 Loss: 1.06905651 time: 0.0030s
Epoch: 6001 Train Accuracy: 0.4776 Test Accuracy: 0.5209 Loss: 1.05804741 time: 0.0030s
Epoch: 7001 Train Accuracy: 0.4776 Test Accuracy: 0.5209 Loss: 1.05804741 time: 0.0040s
Epoch: 8001 Train Accuracy: 0.4776 Test Accuracy: 0.5209 Loss: 1.05804741 time: 0.0030s
Epoch: 9001 Train Accuracy: 0.4807 Test Accuracy: 0.5116 Loss: 1.05660546 time: 0.0050s
Epoch: 10001 Train Accuracy: 0.4807 Test Accuracy: 0.5116 Loss: 1.05660546 time: 0.0040s
Optimizat

In [14]:
d = {'Mask Percentage': ["30/10"], 'Train Initial Accuracy': init, 'Train Final Accuracy': final, 'Test Initial Accuracy': test_init, 'Test Final Accuracy': test_final}

In [15]:
df = pd.DataFrame(data=d)

In [16]:
df

,Mask Percentage,Train Initial Accuracy,Train Final Accuracy,Test Initial Accuracy,Test Final Accuracy
0,30/10,0.131376,0.496136,0.131376,0.502326
